In [2]:
pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


https://www.psycopg.org/docs/
http://localhost:8082

Télécharger le pilote de H2**Relancez le kernel**: Kernel -> Restart kernel ...

Télécharger le pilote de H2

In [3]:
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/postgresql-42.6.0.jar -O postgresql-42.6.0.jar
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/h2-2.1.214.jar -O h2-2.1.214.jar

--2024-04-12 10:31:15--  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/postgresql-42.6.0.jar
Resolving nuage.lip6.fr (nuage.lip6.fr)... 132.227.201.11
Connecting to nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1081604 (1,0M) [application/octet-stream]
Saving to: ‘postgresql-42.6.0.jar’

postgresql-42.6.0.j 100%[===================>]   1,03M  --.-KB/s    in 0,01s   

2024-04-12 10:31:15 (107 MB/s) - ‘postgresql-42.6.0.jar’ saved [1081604/1081604]

--2024-04-12 10:31:16--  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/h2-2.1.214.jar
Resolving nuage.lip6.fr (nuage.lip6.fr)... 132.227.201.11
Connecting to nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2543011 (2,4M) [application/octet-stream]
Saving to: ‘h2-2.1.214.jar’

h2-2.1.214.jar      100%[===================>]   2,42M  --.-KB/s    in 0,03s   

2024-04-12 10:31:1

In [4]:
import psycopg2
import os
local_dir = "./data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

['jo5613.mv.db']

## Démarrer le serveur SQL H2

In [5]:
#Remplacez les valeurs des variables suivantes
#NN= numéro de groupe: chaine de caractères (ex: 01, 02,.., 13), remplacer xx 
NN = '06'
#BB=numero binome: chaîne de caractères (assigné par l'enseignant, ex: 01,..., 15), remplacer xx
BB = '13'
port = 5000+int(NN+BB)
print(f'Le numero du port utilisé pour la connexion à la BD est: {port}')

Le numero du port utilisé pour la connexion à la BD est: 5613


In [6]:
%%bash --bg --out output -s "$port"
java -Dh2.bindAddress=127.0.0.1 -cp h2-2.1.214.jar:postgresql-42.6.0.jar org.h2.tools.Server -pg -pgPort $1 -baseDir ./data -ifNotExists

# Fonctions utiles

## connect_H2

In [7]:
def connect_H2(db,user,password):
    global connection
    try:
        connection
    except:
        connection = None
    if connection != None:
        try:
            connection.close()
            print("Connection closed")
        except  Error as e:
            print(f"The error '{e}' occurred")
    try:
        # connection = sqlite3.connect(path,isolation_level='DEFERRED')
        connection = psycopg2.connect(f"dbname={db} user={user} password={password} host=localhost port={port}")
        print("Connection to H2 DB successful")
    except Exception as e:
        print(f"The error '{e}' occurred")
    return connection

## execute

In [8]:
def execute(connection, query, show=True,close=True):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        print(cursor.rowcount,"rows")
        if show and cursor.rowcount:
            names = [desc[0] for desc in cursor.description]
            for attr in names:
                print(str(attr[:16]).ljust(16),end='|')
            print()
            for attr in names:
                print(''.ljust(17,'-'),end='')
            print()
            for ligne in cursor:
                for attr in ligne:
                    print(str(attr)[:16].ljust(16),end='|')
                print()
        if close:
            cursor.close()
    except Exception as e:
        print(f"The error '{e}' occurred")
        cursor=None
    return cursor

**TD 5 et TME5 : JOINTURES ET IMBRICATION AVEC IN ET EXISTS**

Ce TD/TME utilise les données contenues dans le fichier **bd_jo_v2_H2.sql**


**Attention**: vous pouvez cliquer sur les 3 lignes dans la fenêtre de gauche pour d'afficher les différentes sections du notebook   

# TD5 

On considère le schéma de la base JEUXOLYMPIQUE2014 qui décrit les athlètes et leurs résultats aux
épreuves des sports des Jeux Olympiques d'Hiver Sotchi 2014 :

**PAYS** ( <u>CODEPAYS</u>, NOMP)<br/>
Ex. ('FRA', 'France')<br/>
**SPORT** ( <u>SID</u>, NOMSP)<br/>
Ex. (1, 'Biathlon')<br/>
**EPREUVE** ( <u>EPID</u>, SID*, NOMEP, CATÉGORIE, DATEDEBUT, DATEFIN)<br/>
Ex. (10, 1, 'relais 4x7,5km', 'Hommes', 22/02/2014, 22/02/2014)<br/>
**ATHLETE** ( <u>AID</u>, NOMATH, PRENOMATH, DATENAISSANCE, CODEPAYS*)<br/>
Ex. (1000, 'SOBOLEV', 'Alexey', NULL, 'RUS')<br/>
**EQUIPE** ( <u>EQID</u>, CODEPAYS*)<br/>
Ex. (30, 'SUI')<br/>
**ATHLETESEQUIPE** ( <u>EQID*, AID*</u>)<br/>
Ex. (30, 796) : L'athlète (aid=796) a participé à l'équipe (eqid=30)<br/>
**RANGINDIVIDUEL** ( <u>EPID*, AID*</u>, RANG)<br/>
Ex. (15, 61, 1) : L'athlète (aid=61) a gagné la médaille d'or (rang=1) de l'épreuve (epid=15)<br/>
**RANGEQUIPE** ( <u>EPID*, EQID*</u>, RANG)<br/>
Ex. (10, 30, 14) : L'équipe (eqid=30) a été classée 14e à l'épreuve (epid=10)<br/>


Les attributs qui forment la clé primaire de chaque relation sont soulignés. Les clés étrangères sont
signalées avec une *. Les attributs aid, epid, eqid et sid correspondent aux identifiants des athlètes,
épreuves, équipes et sports et sont utilisés à la fois comme clé primaire ou comme référence (clé
étrangère) vers la relation correspondante.
La relation **PAYS** contient le code et le nom de tous les pays, même si ils n'ont pas participé aux
Jeux Olympiques. Les sports (n-uplets de la relation **SPORT**) sont un ensemble d'épreuves (n-uplets
de la relation **EPREUVE**). Pour chaque épreuve on connaît son nom et les date de début et fin de
l'épreuve. Les épreuves peuvent être individuelles ou par équipe. Dans le premier cas, la
participation des athlètes (n-uplets de la relation ATHLETE) est stocké dans la table
**RANGINDIVIDUEL** qui contient en plus le rang qu'ils ont obtenu (1 pour la médaille d'or). Pour les
épreuves par équipe les résultats sont stockés dans la relation **RANGEQUIPE**, alors que l'information
sur le pays de chaque équipe et ses participants et stocké dans les relations **EQUIPE** et
**ATHLETESEQUIPE**. Dans les relations **RANGINDIVIDUEL** et **RANGEQUIPE** l'attribut rang est égal à
null si l'athlète ou l'équipe a été disqualifié.


## Connexion à H2

In [9]:
connection = connect_H2("jo"+f'{port}',"ba","ba")

Connection to H2 DB successful


## Créer les tables et charger les données bd_jo_v2_H2.sql

In [10]:
schemafile=open("bd-jo-v2_H2.sql",mode="r",encoding='utf-8')
create_schema=schemafile.read()
execute(connection,"drop all objects")
execute(connection, create_schema)
connection.commit()

0 rows
0 rows


## Requêtes

### **Q1**. Les athlètes français (nom pays = 'France') (104 lignes).

In [11]:
query="""
select nomath, prenomath
from athlete
where codepays = (select codepays from pays where nomp = 'France')

"""
cursor=execute(connection,query,show=True)


104 rows
nomath          |prenomath       |
----------------------------------
FOURCADE        |Martin          |
LAMY CHAPPUIS   |Jason           |
VAULTIER        |Pierre          |
CHAPUIS         |Jean Frederic   |
BRUNET          |Marie Laure     |
DORIN HABERT    |Marie           |
ANTHONIOZ       |Deborah         |
MISSILLIER      |Steve           |
BOVOLENTA       |Arnaud          |
MARTINOD        |Marie           |
GAILLARD        |Jean Marc       |
DE LE RUE       |Paul-Henri      |
PINTURAULT      |Alexis          |
BEATRIX         |Jean Guillaume  |
DUVILLARD       |Robin           |
MANIFICAT       |Maurice         |
PERRILLAT BOITEU|Ivan            |
JOSSERAND       |Marion          |
RAMOIN          |Tony            |
TRESPEUCH       |Chloe           |
MATTEL          |Coline          |
MIDOL           |Jonathan        |
ROLLAND         |Kevin           |
LACROIX         |Sebastian       |
LIZEROUX        |Julien          |
JOUBERT         |Brian           |
CHATAIGNIER

### **Q2** . Les épreuves (sport, nom d'épreuve, catégorie) triées par nom de sport, puis par nom d'épreuve dans l'ordre inverse du dictionnaire (98 lignes).

In [12]:
query="""
select nomsp, nomep, categorie
from sport, epreuve
where sport.sid = epreuve.sid
order by nomsp desc, nomep desc
"""

cursor=execute(connection,query,show=True)

98 rows
nomsp           |nomep           |categorie       |
---------------------------------------------------
Surf des neiges |Snowboard cross |Femmes          |
Surf des neiges |Snowboard cross |Hommes          |
Surf des neiges |Slopestyle      |Femmes          |
Surf des neiges |Slopestyle      |Hommes          |
Surf des neiges |Slalom parallèle|Femmes          |
Surf des neiges |Slalom parallèle|Hommes          |
Surf des neiges |Slalom géant par|Femmes          |
Surf des neiges |Slalom géant par|Hommes          |
Surf des neiges |Halfpipe        |Femmes          |
Surf des neiges |Halfpipe        |Hommes          |
Ski de fond     |Sprint par équip|Femmes          |
Ski de fond     |Sprint par équip|Hommes          |
Ski de fond     |Sprint 1,5km    |Femmes          |
Ski de fond     |Sprint 1,5km    |Hommes          |
Ski de fond     |Skiathlon (7,5km|Femmes          |
Ski de fond     |Skiathlon (15km |Hommes          |
Ski de fond     |Relais 4x5km    |Femmes          |
Ski 

### **Q3** . Les athlètes ayant participé à une équipe de France (30 lignes).

In [17]:
query=""" 
select  distinct a.nomath, a.prenomath
from athlete a, athletesequipe ae, equipe e
where a.aid = ae.aid and ae.eqid = e.eqid and e.codepays = (select codepays from pays where nomp = 'France')
"""

cursor=execute(connection,query,show=True)

30 rows
nomath          |prenomath       |
----------------------------------
AYMONIER        |Celia           |
Alexis GODEFROY |Thibault        |
BAILLARD        |Jeremy          |
BEATRIX         |Jean Guillaume  |
BESCOND         |Anais           |
BOEUF           |Alexis          |
BOURZAT         |Fabian          |
BOUTHERIN       |Jeremie         |
BRAUD           |Francois        |
CARRON          |Pernelle        |
COSTERG         |Loic            |
DESTHIEUX       |Simon           |
DORIN HABERT    |Marie           |
DUVILLARD       |Robin           |
Daniel Jean-Paul|Vincent         |
FAIVRE PICON    |Anouk           |
FOURCADE        |Martin          |
GAILLARD        |Jean Marc       |
HEINRICH        |Romain          |
HUGUE           |Coraline        |
JEAN            |Aurore          |
JONES           |Lloyd           |
LACROIX         |Sebastian       |
LAHEURTE        |Maxime          |
LAMY CHAPPUIS   |Jason           |
LEFORT          |Elly            |
MANIFICAT   

### **Q4**. Les épreuves (sport, épreuve, catégorie) auxquelles participent des équipes (25 lignes).

In [16]:
query="""
select distinct nomsp, nomep, categorie
from sport, epreuve, rangEquipe
where sport.sid = epreuve.sid and rangEquipe.epid = epreuve.epid
"""

cursor=execute(connection,query,show=True)

25 rows
nomsp           |nomep           |categorie       |
---------------------------------------------------
Biathlon        |Relais mix      |Mixte           |
Biathlon        |relais 4x6km    |Femmes          |
Biathlon        |relais 4x7,5km  |Hommes          |
Bobsleigh       |bob à deux      |Femmes          |
Bobsleigh       |bob à deux      |Hommes          |
Bobsleigh       |bob à quatres   |Hommes          |
Combiné nordique|Par équipes     |Hommes          |
Curling         |curling         |Femmes          |
Curling         |curling         |Hommes          |
Hockey sur glace|hockey sur glace|Femmes          |
Hockey sur glace|hockey sur glace|Hommes          |
Luge            |Double          |Mixte           |
Luge            |Relais par équip|Mixte           |
Patinage artisti|Couples         |Mixte           |
Patinage artisti|Danse sur glace |Mixte           |
Patinage artisti|Par équipes     |Mixte           |
Patinage de vite|Poursuite par éq|Femmes          |
Pati

### **Q5**. Le pays qui a gagné, en équipe, la médaille d'or dans l'épreuve de la catégorie 'Femmes' intitulée 'relais 4x6km' du sport 'Biathlon' (Ukraine).

In [19]:
query="""
select nomp
from pays p, sport s, epreuve ep, rangEquipe rEq, equipe eq
where s.nomsp = 'Biathlon' and s.sid = ep.sid and ep.nomep = 'relais 4x6km' and ep.categorie = 'Femmes' and rEq.epid = ep.epid and rEq.rang=1 and rEq.eqid = eq.eqid and eq.codepays=p.codepays
"""

cursor=execute(connection,query,show=True)

1 rows
nomp            |
-----------------
Ukraine         |


### **Q6** . Les athlètes ayant participé à au moins une épreuve individuelle et au moins une par équipe (372 lignes).

In [29]:
query="""
select distinct a.nomath, a.prenomath
from athlete a, rangIndividuel rIn
where a.aid = rIn.aid and a.aid = any (select a2.aid from athlete a2, rangEquipe rEq, ATHLETESEQUIPE aEq where rEq.eqid = aEq.eqid and aEq.aid = a2.aid)

"""

cursor=execute(connection,query,show=True)

372 rows
nomath          |prenomath       |
----------------------------------
ABRAMENKO       |Evgeny          |
AHONEN          |Janne           |
ALEXANDER       |Nicholas        |
AN              |Victor          |
ANEV            |Krasimir        |
ANGERER         |Tobias          |
ANTOLEC         |Jan             |
ANTSYBOR        |Maryna          |
ANTYPENKO       |Tetyana         |
ARWIDSON        |Tobias          |
AYMONIER        |Celia           |
BABIKOV         |Ivan            |
BACHLEDA-CURUS  |Katarzyna       |
BAILEY          |Lowell          |
BARDAL          |Anders          |
BAUER           |Armin           |
BAUER           |Klemen          |
BAUER           |Lukas           |
BAUMANN         |Jonas           |
BEATRIX         |Jean Guillaume  |
BERGER          |Tora            |
BERGMAN         |Carl Johan      |
BESCOND         |Anais           |
BESSMERTNYKH    |Alexander       |
BIELER          |Christoph       |
BJOERGEN        |Marit           |
BJOERNDALEN

### **Q7** . Les homonymes (les nom de familles portés par deux athlètes ou plus) (141 lignes)

In [31]:
query="""
select distinct a1.nomath
from athlete a1, athlete a2
where a1.nomath = a2.nomath and a1.aid <> a2.aid
"""

cursor=execute(connection,query,show=True)

141 rows
nomath          |
-----------------
ABRAMENKO       |
ANDERSON        |
BAUER           |
BAUMANN         |
BECKERT         |
BJORNSEN        |
BOE             |
BOWMAN          |
BRAATEN         |
BROWN           |
BROZ            |
Benz            |
CALDWELL        |
CAVIEZEL        |
CHOI            |
COLOGNA         |
COOK            |
COSTA           |
CRAWFORD        |
CURRIER         |
DAVIS           |
DE JONG         |
DUFOUR-LAPOINTE |
DUKURS          |
DVORAK          |
EDER            |
EGGER           |
Ericsson        |
FAIVRE          |
FANCHINI        |
FERNANDEZ       |
FISCHER         |
FISCHNALLER     |
FLETCHER        |
FONTANA         |
FOURCADE        |
GAGNON          |
GAILLARD        |
GASPARIN        |
GASSER          |
GISIN           |
GOLD            |
GREGG           |
GRUBER          |
GUTIERREZ       |
HALLER          |
HAMELIN         |
HAMILTON        |
HANSEN          |
HARVEY          |
HERNANDEZ       |
HOFER           |
INNERHOFER      |
I

### **Q8**. Les athlètes ayant participé à (au moins) deux épreuves individuelles (706 lignes).

In [39]:
query="""
select distinct a.nomath, a.prenomath
from athlete a, rangIndividuel r1, rangIndividuel r2
where a.aid = r1.aid and r1.aid = r2.aid and r1.epid <> r2.epid
"""

cursor=execute(connection,query,show=True)

706 rows
nomath          |prenomath       |
----------------------------------
ABDERHALDEN     |Marianne        |
ABOLTINA        |Agnese          |
ABRAMASHVILI    |Iason           |
ABRAMENKO       |Evgeny          |
ACHIRILOAIE     |Ioan Valeriu    |
AGREITER        |Debora          |
AHONEN          |Janne           |
AKHMADIYEV      |Yerdos          |
ALCOTT          |Chimene         |
ALESSANDRIA     |Arnaud          |
ALEXANDER       |Nicholas        |
ALMOUKOV        |Alexei          |
AMMANN          |Simon           |
AMMAR           |Amanda          |
AN              |Victor          |
ANDERSSON       |David           |
ANEV            |Krasimir        |
ANGERMUELLER    |Monique         |
ANTSYBOR        |Maryna          |
ANTYPENKO       |Tetyana         |
ARWIDSON        |Tobias          |
ASGEIRSDOTTIR   |Erla            |
AYDOVA          |Yekaterina      |
AYMONIER        |Celia           |
BABENKO         |Dmitry          |
BABIKOV         |Ivan            |
BAILEY     

### **Q9**. Les athlètes ayant gagné une médaille dans une épreuve individuelle, mais ayant été disqualifiés
dans une autre (14 lignes).

In [40]:
query="""
select distinct a.nomath, a.prenomath
from athlete a, rangIndividuel r1, rangIndividuel r2
where a.aid = r1.aid and r1.aid = r2.aid and r1.epid <> r2.epid and r1.rang<4 and r2.rang is null

"""

cursor=execute(connection,query,show=True)

14 rows
nomath          |prenomath       |
----------------------------------
FENNINGER       |Anna            |
GISIN           |Dominique       |
GUT             |Lara            |
HOEFL-RIESCH    |Maria           |
INNERHOFER      |Christof        |
JANSRUD         |Kjetil          |
JIANROU         |Li              |
KOWALCZYK       |Justyna         |
LIGETY          |Ted             |
MANCUSO         |Julia           |
MAYER           |Matthias        |
MISSILLIER      |Steve           |
PINTURAULT      |Alexis          |
WEIBRECHT       |Andrew          |


### **Q10**. Les athlètes ayant gagné (au moins) une médaille dans une épreuve individuelle (182 lignes).

In [41]:
query="""
select distinct a.nomath, a.prenomath
from athlete a, rangIndividuel r1
where a.aid = r1.aid and r1.rang<4

"""

cursor=execute(connection,query,show=True)

182 rows
nomath          |prenomath       |
----------------------------------
AN              |Victor          |
ANDERSON        |Jamie           |
ANTOINE         |Matthew         |
BARDAL          |Anders          |
BEATRIX         |Jean Guillaume  |
BEEK            |Lotte van       |
BERGER          |Tora            |
BERGSMA         |Jorrit          |
BILODEAU        |Alexandre       |
BJOERGEN        |Marit           |
BJOERNDALEN     |Ole Einar       |
BLOKHUIJSEN     |Jan             |
BOER            |Margot          |
BOVOLENTA       |Arnaud          |
BOWMAN          |Maddie          |
BRIGHT          |Torah           |
BRODKA          |Zbigniew        |
CHAN            |Patrick         |
CHAPUIS         |Jean Frederic   |
CHERNOUSOV      |Ilia            |
CHRISTENSEN     |Joss            |
CLARK           |Kelly           |
COLOGNA         |Dario           |
COURNOYER       |Charle          |
DAJING          |Wu              |
DE JONG         |Bob             |
DEIBOLD    

**NOT EXISTS / NOT IN**

### **Q11**.  Les noms de pays qui n’ont qu’une seule équipe (9 lignes).

In [45]:
query="""
select p.nomp
from pays p, equipe eq1
where p.codepays = eq1.codepays and not exists (select * from equipe eq2 where eq2.eqid <> eq1.eqid and eq1.codepays = eq2.codepays)
"""

cursor=execute(connection,query,show=True)

9 rows
nomp            |
-----------------
Azerbaïdjan     |
Belgique        |
Bulgarie        |
Espagne         |
Jamaïque        |
Lituanie        |
Monaco          |
Serbie          |
Turquie         |


### **Q12**. Les athlètes ayant participé à exactement 2 épreuves individuelles (402 lignes).

In [47]:
query="""
select distinct a.nomath, a.prenomath
from athlete a, rangIndividuel r1, rangIndividuel r2
where a.aid = r1.aid and r1.aid = r2.aid and r1.epid <> r2.epid and not exists (select * from rangIndividuel r3 where r3.epid<> r2.epid and r3.epid <> r1.epid and r3.aid = a.aid)
"""

cursor=execute(connection,query,show=True)

402 rows
nomath          |prenomath       |
----------------------------------
ABDERHALDEN     |Marianne        |
ABRAMASHVILI    |Iason           |
ACHIRILOAIE     |Ioan Valeriu    |
AGREITER        |Debora          |
AHONEN          |Janne           |
AKHMADIYEV      |Yerdos          |
ALEXANDER       |Nicholas        |
ALMOUKOV        |Alexei          |
AMMANN          |Simon           |
ANDERSSON       |David           |
ANGERMUELLER    |Monique         |
ANTSYBOR        |Maryna          |
ANTYPENKO       |Tetyana         |
ASGEIRSDOTTIR   |Erla            |
AYMONIER        |Celia           |
BAMFORD         |Emily           |
BANCORA         |Salome          |
BARBU           |Alexandru       |
BARDAL          |Anders          |
BARTHET         |Anne-Sophie     |
BARWOOD         |Adam            |
BAUD            |Adeline         |
BAUER           |Armin           |
BAUER           |Lukas           |
BAUMANN         |Jonas           |
BAUMGAERTNER    |Alexej          |
BECKERT    

# TME5: JOINTURES ET IMBRICATION AVEC IN ET EXISTS 

Ce TME utilise les données contenues dans le fichier **bd_jo_v2_H2.sql**


## Requêtes

### **Q1**. Les sports auxquels LESSER Erik a participé en supposant qu'il a participé aux épreuves
individuelles (Ski de fond, Biathlon).

In [49]:
query="""
select distinct s.nomsp
from sport s, epreuve ep, rangIndividuel r, athlete a
where a.nomath = 'LESSER' and a.prenomath = 'Erik' and r.aid = a.aid and r.epid = ep.epid and ep.sid = s.sid

"""

cursor=execute(connection,query,show=True)

2 rows
nomsp           |
-----------------
Biathlon        |
Ski de fond     |


### **Q2**. Les athlètes ayant participé aux épreuves individuelles de (au moins) deux sports différents ( PETROVIC Milanko, TER MORS Jorien, LESSER Erik, PEIFFER Arnd).

In [51]:
query="""
select distinct a.nomath, a.prenomath
from athlete a, rangIndividuel r, epreuve ep
where a.aid = r.aid and r.epid = ep.epid and ep.sid <> any (select ep2.sid from rangIndividuel r2, epreuve ep2 where a.aid = r2.aid and r2.epid = ep2.epid)
"""

cursor=execute(connection,query,show=True)

4 rows
nomath          |prenomath       |
----------------------------------
LESSER          |Erik            |
PEIFFER         |Arnd            |
PETROVIC        |Milanko         |
TER MORS        |Jorien          |


**NOT EXISTS / NOT IN / IS NULL**

### **Q3**. Les dates de début et de fin des Jeux Olympique 2014 (06/02/2014, 23/02/2014).

In [52]:
query="""
select ep1.datedebut
from epreuve ep1
where ep1.datedebut <= all (select datedebut from epreuve ep2)

Union

select ep1.datefin
from epreuve ep1
where ep1.datefin >= all (select datefin from epreuve ep2)
"""

cursor=execute(connection,query,show=True)

2 rows
datedebut       |
-----------------
2014-02-06      |
2014-02-23      |


### **Q4**. Les pays qui ont des participants de moins de 18 ans ou de plus que 40 au 24-02-2016
(Attention : il faut éliminer les athlètes dont on ne connaît pas la date de naissance) (AUT Autriche, CAN Canada, GER Allemagne, ITA Italie, JPN Japon, NOR Norvège, RUS Russie).

In [62]:
query="""
select distinct p.nomp
from pays p, epreuve ep, athlete a, rangIndividuel r1
where  p.codePays = a.codePays AND a.dateNaissance IS NOT NULL and ep.epid = r1.epid and r1.aid=a.aid and (a.datenaissance > '1998-02-24' or a.datenaissance < '1976-02-24')
"""

cursor=execute(connection,query,show=True)

7 rows
nomp            |
-----------------
Allemagne       |
Autriche        |
Canada          |
Italie          |
Japon           |
Norvège         |
Russie          |


### **Q5**. Les équipes n'ayant pas d'athlète (la base de données ne contient pas l'information sur les participants) (13 lignes)

In [68]:
query="""
select eq.eqid
from equipe eq
where not exists (select * from athletesEquipe aEq where aEq.eqid = eq.eqid)
"""

cursor=execute(connection,query,show=True)

13 rows
eqid            |
-----------------
232             |
160             |
231             |
235             |
162             |
233             |
229             |
163             |
230             |
161             |
159             |
234             |
167             |


### **Q6**. Les équipes qui ont exactement 2 athlètes. Retourner l'équipe en question avec ses deux seuls athlètes (114 lignes).


In [70]:
query="""
select distinct eq.eqid
from equipe eq, athletesEquipe aEq1, athletesEquipe aEq2
where aEq1.eqid = eq.eqid and aEq2.eqid = eq.eqid and aEq1.aid <> aEq2.aid and not exists (select * from athletesEquipe aEq3 where aEq3.eqid= eq.eqid and aEq3.aid <> aEq1.aid and aEq3.aid <> aEq2.aid)
"""

cursor=execute(connection,query,show=True)

114 rows
eqid            |
-----------------
52              |
53              |
54              |
55              |
56              |
57              |
58              |
59              |
60              |
61              |
62              |
63              |
64              |
65              |
66              |
67              |
68              |
69              |
70              |
71              |
72              |
73              |
74              |
75              |
76              |
77              |
78              |
79              |
80              |
81              |
82              |
83              |
84              |
85              |
86              |
87              |
88              |
89              |
90              |
91              |
92              |
93              |
94              |
95              |
96              |
97              |
98              |
99              |
100             |
168             |
169             |
170             |
171             |
1

### **Q7**. Les sports qui n'ont pas d'épreuves de catégorie Mixte (12 lignes).

In [75]:
query="""
select s.nomsp
from sport s
where NOT EXISTS (SELECT * FROM Epreuve e WHERE s.sid = e.sid AND e.categorie = 'Mixte')

"""

cursor=execute(connection,query,show=True)

12 rows
nomsp           |
-----------------
Bobsleigh       |
Combiné nordique|
Curling         |
Hockey sur glace|
Patinage de vite|
Patinage de vite|
Saut à ski      |
Skeleton        |
Ski acrobatique |
Ski alpin       |
Ski de fond     |
Surf des neiges |


### **Q8**. Les athlètes qui ont gagné une médaille d’or (au moins) mais pas de médaille d’argent ni de bronze (53 lignes)

In [77]:
query="""
select distinct a.nomath, a.prenomath
from athlete a, rangIndividuel r
where r.aid = a.aid and r.rang = 1 and not exists (select * from rangIndividuel r2 where r2.aid = a.aid and (r2.rang = 2 or r2.rang = 3))

"""

cursor=execute(connection,query,show=True)

53 rows
nomath          |prenomath       |
----------------------------------
ANDERSON        |Jamie           |
BILODEAU        |Alexandre       |
BJOERGEN        |Marit           |
BJOERNDALEN     |Ole Einar       |
BOWMAN          |Maddie          |
BRODKA          |Zbigniew        |
CHAPUIS         |Jean Frederic   |
CHRISTENSEN     |Joss            |
COLOGNA         |Dario           |
DOMRACHEVA      |Darya           |
DUFOUR-LAPOINTE |Justine         |
DUJMOVITS       |Julia           |
FALLA           |Maiken Caspersen|
FARRINGTON      |Kaitlyn         |
FRENZEL         |Eric            |
GEISENBERGER    |Natalie         |
GISIN           |Dominique       |
GRAABAK         |Joergen         |
GROOTHUIS       |Stefan          |
HAMELIN         |Charles         |
HANYU           |Yuzuru          |
HATTESTAD       |Ola Vigen       |
HONG            |Zhang           |
HOWELL          |Dara            |
JIANROU         |Li              |
KOTSENBURG      |Sage            |
KOWALCZYK   

### **Q9**. Les athlète(s) qui ont fini dernier d’une épreuve individuelle. Indiquer leur nom, prenom, le nom du sport et de l’épreuve et leur rang. Attention : il faut filtrer les athlètes disqualifiés (73 lignes).

In [89]:
query="""
select a.nomath, a.prenomath, s.nomsp, ep.nomep, r.rang
from athlete a, sport s, rangIndividuel r, epreuve ep
where a.aid = r.aid and ep.epid = r.epid and s.sid = ep.sid and r.rang is not null and not exists (select * from rangIndividuel r2 where r2.epid = r.epid and r2.rang > r.rang)

"""

cursor=execute(connection,query,show=True)

73 rows
nomath          |prenomath       |nomsp           |nomep           |rang            |
-------------------------------------------------------------------------------------
DUBAREZAVA      |Nastassia       |Biathlon        |10km poursuite  |56              |
HILDEBRAND      |Franziska       |Biathlon        |12,5km départ gr|28              |
DUBAREZAVA      |Nastassia       |Biathlon        |15km            |56              |
DUBAREZAVA      |Nastassia       |Biathlon        |7,5km           |56              |
KENESHEV        |Dias            |Biathlon        |10km            |87              |
STEPTSENKO      |Danil           |Biathlon        |12,5km poursuite|59              |
SUMANN          |Christoph       |Biathlon        |15km départ grou|27              |
GOMBOS          |Karoly          |Biathlon        |20km            |88              |
KLIMOV          |Evgeniy         |Combiné nordique|Individuel      |45              |
TIIRMAA         |Karl-August     |Combiné nord

# Fermer la connexion

In [90]:
connection.commit() # implicit avec close
connection.close()

In [91]:
connection

<connection object at 0x7f07ec3d1080; dsn: 'user=ba password=xxx dbname=jo5613 host=localhost port=5613', closed: 1>